In [1]:
import sys
import platform
from os.path import join, exists, abspath, dirname
from os import getcwd, makedirs
from glob import glob

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import colormaps, cm
from matplotlib import colors as mcolors
import scipy
import seaborn as sns
from statsmodels.stats.anova import AnovaRM

import h5py

from tqdm import tqdm

In [2]:
dir_current = getcwd().replace('\\','/')

tmp = dir_current.split('/')
idx = [ii for ii, s in enumerate(tmp) if s=='github'][0]

dir_git = '/'.join(tmp[:idx+1])
dir_git

'/home/sungbeenpark/github'

In [3]:
dname = join(dir_git,'SeqSpatialSupp_fMRI')
sys.path.append(dname)
from SSS import deal_spm
from SSS import util as su
from SSS import plot as splt
from SSS import stat as sstat

In [4]:
import warnings

---

In [5]:
mpl.rcParams['font.family'] = 'Helvetica'

### Font를 못 읽어 온다면, matplotlib 캐시 초기화
`rm -rf ~/.cache/matplotlib`

---

In [16]:
list_sn = su.get_list_sn()

In [8]:
dir_root = su.get_dir_root()

In [23]:
fname = join(dir_root,'participants.tsv')
df = pd.read_csv(fname, delimiter='\t', header=0)

In [25]:
pattern = '|'.join(list_sn)
df = df[df.subj_id.str.contains(pattern, na=False)].reset_index(drop=True)
df

,sn,subj_id,sex,age,handedness,site,year,resolution,numTR,TR,numSess,runlist,FuncRawNameSess,AnatRawName,fmapMagnitudeNameSess,fmapPhaseNameSess,locACx,locACy,locACz,good
0,1,S01,m,21,right,UWO,2024,1.8mm-iso,410,1000,1,1.2.3.4.5.6.7.8,sub-S01_task-task_run-XX_bold,sub-S01_acq-MP2RAGE_run-01_T1w,sub-S01_magnitude1,sub-S01_phasediff,75.53900,129.7930,118.3040,1
1,2,S02,m,26,right,KOR,2024,1.8mm-iso,410,1000,1,1.2.3.4.5.6.7.8,sub-S02_task-task_run-XX_bold,sub-S02_acq-MP2RAGE_run-01_T1w,sub-S02_magnitude1,sub-S02_phasediff,88.57500,126.2590,129.8220,1
2,3,S03,f,39,right,KOR,2024,1.8mm-iso,410,1000,1,1.2.3.4.5.6.7.8,sub-S03_task-task_run-XX_bold,sub-S03_acq-MP2RAGE_run-01_T1w,sub-S03_magnitude1,sub-S03_phasediff,78.82800,121.8370,124.5740,1
3,5,S05,m,42,right,KOR,2024,1.8mm-iso,410,1000,1,1.2.3.4.5.6.7.8,sub-S05_task-task_run-XX_bold,sub-S05_acq-MP2RAGE_run-01_T1w,sub-S05_magnitude1,sub-S05_phasediff,87.37190,120.8118,130.0988,1
4,6,S06,f,45,right,KOR,2024,1.8mm-iso,410,1000,1,1.2.3.4.5.6.7.8,sub-S06_task-task_run-XX_bold,sub-S06_acq-MP2RAGE_run-01_T1w,sub-S06_magnitude1,sub-S06_phasediff,82.13506,122.1487,124.3632,1
5,8,S08,m,24,right,KOR/UWO,2024,1.8mm-iso,410,1000,1,1.2.3.4.5.6.7.8.9,sub-S08_task-task_run-XX_bold,sub-S08_acq-MP2RAGE_run-01_T1w,sub-S08_magnitude1,sub-S08_phasediff,86.68162,116.3777,126.8047,1
6,9,S09,f,31,right,KOR/UWO,2024,1.8mm-iso,410,1000,1,1.2.3.4.5.6.7.8.9,sub-S09_task-task_run-XX_bold,sub-S09_acq-MP2RAGE_run-01_T1w,sub-S09_magnitude1,sub-S09_phasediff,79.10100,123.0000,132.0850,1
7,10,S10,f,25,right,KOR/UWO,2024,1.8mm-iso,410,1000,1,1.2.3.4.5.6.7.8.9,sub-S10_task-task_run-XX_bold,sub-S10_acq-MP2RAGE_run-01_T1w,sub-S10_magnitude1,sub-S10_phasediff,75.97858,118.9866,135.6180,1
8,11,S11,f,19,right,KOR,2024,1.8mm-iso,410,1000,1,1.2.3.4.5.6.7.8.9,sub-S11_task-task_run-XX_bold,sub-S11_acq-MP2RAGE_run-01_T1w,sub-S11_magnitude1,sub-S11_phasediff,78.19524,127.8567,120.0954,1
9,12,S12,f,22,right,KOR,2024,1.8mm-iso,410,1000,1,1.2.3.4.5.6.7.8.9,sub-S12_task-task_run-XX_bold,sub-S12_acq-MP2RAGE_run-01_T1w,sub-S12_magnitude1,sub-S12_phasediff,83.92808,121.7979,130.0353,1


---

In [28]:
tmp = df[df.subj_id.str.contains('S', na=False)]

In [30]:
cnt = tmp.sex.value_counts()
cnt

sex
f    7
m    5
Name: count, dtype: int64

In [34]:
age_mean = df.age.mean()
age_std = df.age.std()
print('mean age: %.1f yr\tstd age: %.1f yr'%(age_mean, age_std))

mean age: 27.9 yr	std age: 9.0 yr


---